# Export BSCCM from older more messy format

In [210]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import zarr
import matplotlib.gridspec as gridspec
from skimage.transform import rescale, resize, downscale_local_mean
import os
import json

COHERENT = False



home = str(Path.home())

if COHERENT:
    data_root = home + '/leukosight_data/final_filtered_crops_single/'
    dataset = zarr.open(data_root + 'crops_single.zarr', 'r')
    csv_path = data_root + 'crops_fieldstop_filtered_cleaned_record_single.csv'
else:
    data_root = home + '/leukosight_data/final_filtered_crops/'
    dataset = zarr.open(data_root + 'crops_fieldstop_filtered.zarr', 'r')
    rgb_dataset = zarr.open(data_root + 'rgb_crops.zarr', 'r')
    csv_path = data_root + 'crops_record_histology_matched.csv'
    
dataframe = pd.read_csv(csv_path, index_col=0, low_memory=False)
channel_names = dataset[list(dataset)[0]]['all_blobs'].attrs['channel_names']
slidescan_db = pd.read_csv('slidescan_db.csv')

all_filenames =  ['{}/{}'.format(int(dataset_index), crop_name) for dataset_index, crop_name 
         in zip(dataframe['dataset_index'].to_numpy(), dataframe['blob_name'].tolist())]


channel_names = dataset[list(dataset)[0]]['all_blobs'].attrs['channel_names']
fluor_channel_names = ['BV421', 'BV510', 'BV570', 'BV605', 'BV650', 'BV711']
markers = ['CD123', 'CD3','CD19', 'CD56', 'HLA-DR', 'CD45', 'CD14', 'CD16']
merged_markers = ['CD123', 'CD3/CD19/CD56',
       'HLA-DR', 'CD45', 'CD14', 'CD16', 'autofluor_1', 'autofluor_2']

fl_channel_renaming = {
    'F1_BV711': 'Fluor_690-',
    'F2_BV650': 'Fluor_627-673',
    'F3_BV605': 'Fluor_585-625',
    'F4_BV570': 'Fluor_550-570',
    'F5_BV510': 'Fluor_500-550',
    'F6_BV421': 'Fluor_426-446',
}

## Process dataframe and resave

In [215]:
df_new = dataframe.copy()
#remove stuff for old zarr structure and add in new

#copy in metadata specific to slides
for marker in markers:
    print(marker)
   
    #also add in other data while you're at it
    for ds_index in slidescan_db['Index'].tolist():
        df_new.loc[df_new['dataset_index'] == ds_index, 
                      'batch'] = slidescan_db[slidescan_db['Index'] == ds_index]['Batch'].tolist()[0]
        df_new.loc[df_new['dataset_index'] == ds_index, 
                      'antibodies'] = slidescan_db[slidescan_db['Index'] == ds_index]['Marker'].tolist()[0]
        df_new.loc[df_new['dataset_index'] == ds_index, 
                      'imaging_date'] = slidescan_db[slidescan_db['Index'] == ds_index]['Imaging date'].tolist()[0]
        
        if ds_index in [0, 1, 2, 3, 4, 5, 6, 7]:
            df_new.loc[df_new['dataset_index'] == ds_index, 
                      'shading_correction_index'] = 0
        elif ds_index in [8, 9, 10, 11, 12, 13, 14, 15, 16]:
            df_new.loc[df_new['dataset_index'] == ds_index, 
                      'shading_correction_index'] = 1
        else:
            df_new.loc[df_new['dataset_index'] == ds_index, 
                      'shading_correction_index'] = 2
        
    df_new.loc[df_new['dataset_index'] == 15, 'notes'] = 'CD19 antibody conc. 35% of others'
    df_new.loc[df_new['dataset_index'] == 24, 'notes'] = 'CD19 antibody conc. 35% of others'

#change data type    
df_new['antibodies'] = df_new['antibodies'].astype(str)
df_new['notes'] = df_new['notes'].astype(str)
df_new['imaging_date'] = pd.to_datetime(df_new['imaging_date'])

    
#Slide replicate aka slide_index
if not COHERENT:
    #change batch to 0 indexing
    df_new.batch -= 1
    slide_index = df_new.dataset_index.map(lambda x: 1 if (x == 24 or x == 26)  else 0)
    df_new['data_path'] = 'antibodies_' + df_new.antibodies + \
            '/batch_'  + df_new.batch.map(lambda x: str(int(x))) + \
            '/slide_replicate_' + slide_index.astype(str)
    df_new['slide_replicate'] = slide_index
else:
    #change batch to 0 indexing
    df_new['data_path'] = 'antibodies_' + df_new.antibodies + \
            '/batch_1' + \
            '/slide_replicate_0' 
    df_new['slide_replicate'] = 0
    df_new['batch'] = 0

lf_source_paths = (df_new['dataset_index'].astype(int).astype(str) + '/' + df_new['blob_name']).to_list()


if not COHERENT:
    df_new['histology_dataset_name']
    df_new['closest_histology_cell_name']
    df_new = df_new.rename(columns = {"histology_match": "has_matched_histology_cell"})
        
    #get the paths to histology data in the other dataset
    # df_has_hist = df_new[df_new.has_matched_histology_cell == True]
    hist_source_paths = (df_new['histology_dataset_name'] + '/' + df_new['closest_histology_cell_name']).to_list()

    # df_new.loc[df_new.has_matched_histology_cell == True, 'hist_path'] = hist_path
    # hist_paths = df_new['hist_path'].to_list()
    # [h for h in hist_paths if type(h) == str]
    #now delete intermediate hist metadat
    del df_new['histology_dataset_name']
    del df_new['closest_histology_cell_name']


df_new = df_new.rename(columns = {
    "blob_x": "position_in_fov_x_pix",
    "blob_y": "position_in_fov_y_pix",
    "blob_radius": "detection_radius",
#     "affine_corrected_target": "batch_corrected_fluor",
    "shading_correction_index": "fluor_shading_correction_index",
                        })
#delete a bunch of stuff
for name in ['blob_name',
            'dataset_index',
            'closest_cell',
            'closest_cell_edge',
            'jiggler_shifts',
            'valid_fluor',
            'Positive_cells',
            'hist_gradient',
            'position_index',
            'closest_histology_cell_dist',
            '2nd_closest_histology_cell_name',
            '2nd_closest_histology_cell_brightness',
            '2nd_closest_histology_cell_dist']:
    if name in df_new:
        del df_new[name]
    
#reset global index
df_new['global_index'] = np.arange(len(df_new))

#delete fluor stuff. will re add later
for name in df_new.columns:
    if 'unmixed' in name or 'BV' in name or 'autofluor' in name:
        del df_new[name]


CD123
CD3
CD19
CD56
HLA-DR
CD45
CD14
CD16


## Process Zarr data and resave

In [233]:
#dimensions for subsampling crops
crop_dim = 128
remove_size = (150 - crop_dim)
min_extent = remove_size // 2
max_extent = -remove_size // 2

if not COHERENT:
    hist_crop_dim = round(128 * 0.166 / 0.0534)
    path = [x for x in hist_source_paths if type(x) == str][0]
    hist_remove_COHERHENTze = (rgb_dataset[path].shape[1] - hist_crop_dim)
    hist_min_extent = hist_remove_size // 2
    hist_max_extent = -hist_remove_size // 2
    new_name = 'BSCCM_images.zarr'
else:
    new_name = 'BSCCM-coherent_images.zarr'


home = str(Path.home())
if COHERENT:
    export_root = home + '/leukosight_data/BSCCM-coherent/'
else:
    export_root = home + '/leukosight_data/BSCCM/'

channel_names = dataset[list(dataset)[0]]['all_blobs'].attrs['channel_names']
if not COHERENT:
    led_array_channel_names = channel_names[:22] + [channel_names[-3]]
    fluor_channel_names = channel_names[22:28]
else:
    led_array_channel_names = ['led_{}'.format(i) for i in range(600) if 'led_{}'.format(i)  in channel_names]
    fluor_channel_names = [c for c in channel_names if 'BV' in c]
    
new_file = zarr.open(export_root + new_name, mode='w')


for i in range(len(df_new)):
    print(str(i) + ' of  {}\r'.format(len(dataframe)), end='')

    entry = df_new.iloc[i]    
    
    base_path = entry['data_path']
    fluor_dest = base_path + '/fluor/cell_{}'.format(i)
    lf_dest = base_path + '/led_array/cell_{}'.format(i)
    dpc_dest = base_path + '/dpc/cell_{}'.format(i)
    hist_dest = base_path + '/histology/cell_{}'.format(i)
    
    lf_source = lf_source_paths[i]
    if not COHERENT and entry['has_matched_histology_cell']:
        histology_source = hist_source_paths[i]
    else:
        histology_source = None
    
    if not COHERENT:
        fluor_data = dataset[lf_source][22:28, min_extent:max_extent, min_extent:max_extent]
        led_array_data = np.concatenate([dataset[lf_source][:22],
                                         dataset[lf_source][-3][None, ...]])[:, min_extent:max_extent, min_extent:max_extent]
        dpc_data = dataset[lf_source][-1][None, ...][:, min_extent:max_extent, min_extent:max_extent]
    else:
        raw_crop = dataset[lf_source][..., min_extent:max_extent, min_extent:max_extent]
        led_array_data = np.stack([raw_crop[channel_names.index(c)] for c in led_array_channel_names])
        fluor_data = np.stack([raw_crop[channel_names.index(c)] for c in fluor_channel_names])
        dpc_data = dataset[lf_source][-1][None, ...][:, min_extent:max_extent, min_extent:max_extent]
    
    new_file.create_dataset(name=fluor_dest, data=fluor_data.astype(np.uint16), chunks=(1, crop_dim, crop_dim))
    new_file.create_dataset(name=lf_dest, data=led_array_data.astype(np.uint16), chunks=(1, crop_dim, crop_dim))
    new_file.create_dataset(name=dpc_dest, data=dpc_data, chunks=(1, crop_dim, crop_dim))
    
    if histology_source is not None:
#         print(hist_dest)
        hist_data = rgb_dataset[histology_source][..., 
                                    hist_min_extent:hist_max_extent, hist_min_extent:hist_max_extent]
        #reorder to rgb from bgr
        hist_data = np.moveaxis(hist_data, 0, -1)
        new_file.create_dataset(name=hist_dest, data=hist_data, chunks=None)
        
#     if i == 10:
#         break


In [234]:
if COHERENT:
    df_new.to_csv(export_root + 'BSCCM-coherent_index_no_fluor.csv', index=False)
else:
    df_new.to_csv(export_root + 'BSCCM_index_no_fluor.csv', index=False)

## BSCCM global metadata


In [236]:
fl_channel_renaming = {
    'F1_BV711': 'Fluor_690-',
    'F2_BV650': 'Fluor_627-673',
    'F3_BV605': 'Fluor_585-625',
    'F4_BV570': 'Fluor_550-570',
    'F5_BV510': 'Fluor_500-550',
    'F6_BV421': 'Fluor_426-446',
}

if not COHERENT:
    metadata = {
        'led_array': {
            'channel_names': led_array_channel_names,
            'channel_indices': {ch_name: i for i, ch_name in enumerate(led_array_channel_names)},
            'exposure_ms': {
                'Brightfield': 8,
                'DF_50': 29,
                'DF_50_Bottom': 58,
                'DF_50_Right': 58,
                'DF_55': 46,
                'DF_60': 62,
                'DF_60_Bottom': 124,
                'DF_60_Right': 124,
                'DF_65': 79,
                'DF_70': 84,
                'DF_70_Bottom': 168,
                'DF_70_Right': 168,
                'DF_75': 93,
                'DF_80': 142,
                'DF_80_Bottom': 284,
                'DF_80_Right': 284,
                'DF_85': 228,
                'DF_90': 510,
                'DPC_Bottom': 17,
                'DPC_Left': 17,
                'DPC_Right': 17,
                'DPC_Top': 17,
                'LED119': 200
            },
            'wavelength_nm': 515,
            'pixel_size_um': 0.166,
            'objective': {'NA': 0.5, 'magnification': 20}
        },
        'fluorescence': {
            'channel_names': [fl_channel_renaming[f] for f in fluor_channel_names],
            'channel_indices': {fl_channel_renaming[ch_name]: i for i, ch_name
                                in enumerate(fluor_channel_names)},
            'exposure_ms': {
                fl_channel_renaming['F1_BV711']: 1600,
                fl_channel_renaming['F2_BV650']: 2000,
                fl_channel_renaming['F3_BV605']: 1500,
                fl_channel_renaming['F4_BV570']: 1200,
                fl_channel_renaming['F5_BV510']: 1000,
                fl_channel_renaming['F6_BV421']: 600
            },
            'emission_wavelength_nm': {
              'Fluor_690-': [690, None],
              'Fluor_627-673': [627, 673],
              'Fluor_585-625': [585, 625],
              'Fluor_550-570': [550, 570],
              'Fluor_500-550': [500, 550],
              'Fluor_426-446': [426, 446],
            },
            'excitation_wavelength_nm': 405,
            'pixel_size_um': 0.166,
            'objective': {'NA': 0.5, 'magnification': 20}

        },
        'histology': {
            'channel_names': ['Red', 'Green', 'Blue', ],
            'channel_exposures': {'Red': 55, 'Green': 60, 'Blue': 70},
            'wavelength_nm': {'Red':630,  'Green': 515, 'Blue': 460},
            'pixel_size_um': 0.0534,
            'objective': {'NA': 1.4, 'magnification': 63}
        }
    }




    with open(export_root + '/BSCCM_global_metadata.json', 'w') as json_file:
        json.dump(metadata, json_file)
        
else:
    
    metadata = {
        'led_array': {
            'channel_names': led_array_channel_names,
            'channel_indices': {ch_name: i for i, ch_name in enumerate(led_array_channel_names)},
            'exposure_ms': {
               'led_1': 50,
                'led_2': 50,
                'led_5': 50,
                'led_6': 50,
                'led_7': 50,
                'led_8': 50,
                'led_9': 50,
                'led_10': 50,
                'led_13': 50,
                'led_14': 50,
                'led_15': 50,
                'led_16': 50,
                'led_17': 50,
                'led_18': 50,
                'led_19': 50,
                'led_20': 50,
                'led_21': 50,
                'led_22': 50,
                'led_23': 50,
                'led_24': 50,
                'led_25': 50,
                'led_26': 50,
                'led_29': 50,
                'led_30': 50,
                'led_31': 50,
                'led_32': 50,
                'led_33': 50,
                'led_34': 50,
                'led_35': 50,
                'led_36': 50,
                'led_37': 50,
                'led_38': 50,
                'led_39': 50,
                'led_40': 50,
                'led_41': 50,
                'led_42': 50,
                'led_43': 50,
                'led_44': 50,
                'led_45': 50,
                'led_46': 50,
                'led_49': 50,
                'led_50': 50,
                'led_51': 50,
                'led_52': 50,
                'led_53': 50,
                'led_54': 50,
                'led_55': 50,
                'led_56': 50,
                'led_57': 50,
                'led_58': 50,
                'led_59': 50,
                'led_60': 50,
                'led_61': 50,
                'led_62': 50,
                'led_63': 50,
                'led_64': 50,
                'led_65': 50,
                'led_66': 50,
                'led_67': 50,
                'led_68': 50,
                'led_69': 50,
                'led_70': 50,
                'led_71': 50,
                'led_72': 50,
                'led_73': 50,
                'led_74': 50,
                'led_75': 50,
                'led_76': 50,
                'led_77': 50,
                'led_78': 50,
                'led_79': 50,
                'led_80': 50,
                'led_81': 50,
                'led_82': 50,
                'led_85': 50,
                'led_86': 50,
                'led_87': 50,
                'led_88': 50,
                'led_89': 50,
                'led_90': 50,
                'led_91': 50,
                'led_92': 50,
                'led_93': 50,
                'led_94': 50,
                'led_95': 50,
                'led_96': 50,
                'led_97': 50,
                'led_98': 50,
                'led_99': 50,
                'led_100': 50,
                'led_101': 50,
                'led_102': 50,
                'led_103': 50,
                'led_104': 50,
                'led_105': 50,
                'led_106': 50,
                'led_107': 50,
                'led_108': 50,
                'led_109': 50,
                'led_110': 50,
                'led_111': 50,
                'led_112': 50,
                'led_113': 50,
                'led_114': 50,
                'led_115': 50,
                'led_116': 50,
                'led_117': 50,
                'led_118': 50,
                'led_119': 50,
                'led_121': 50,
                'led_122': 50,
                'led_123': 50,
                'led_124': 50,
                'led_125': 50,
                'led_126': 50,
                'led_127': 50,
                'led_128': 50,
                'led_129': 50,
                'led_130': 50,
                'led_131': 50,
                'led_132': 50,
                'led_133': 50,
                'led_134': 50,
                'led_135': 50,
                'led_136': 50,
                'led_137': 50,
                'led_138': 50,
                'led_139': 50,
                'led_140': 50,
                'led_141': 50,
                'led_142': 50,
                'led_143': 50,
                'led_144': 50,
                'led_145': 50,
                'led_146': 50,
                'led_147': 50,
                'led_148': 50,
                'led_149': 50,
                'led_150': 50,
                'led_151': 50,
                'led_152': 50,
                'led_153': 50,
                'led_154': 50,
                'led_155': 50,
                'led_156': 50,
                'led_157': 50,
                'led_158': 50,
                'led_159': 50,
                'led_160': 50,
                'led_161': 50,
                'led_162': 50,
                'led_163': 50,
                'led_164': 50,
                'led_165': 50,
                'led_166': 50,
                'led_167': 50,
                'led_168': 50,
                'led_169': 50,
                'led_170': 50,
                'led_171': 50,
                'led_172': 50,
                'led_173': 50,
                'led_174': 50,
                'led_175': 50,
                'led_176': 50,
                'led_177': 50,
                'led_178': 50,
                'led_179': 50,
                'led_180': 50,
                'led_181': 50,
                'led_182': 50,
                'led_183': 50,
                'led_184': 50,
                'led_185': 50,
                'led_186': 50,
                'led_189': 50,
                'led_190': 50,
                'led_191': 50,
                'led_192': 50,
                'led_193': 50,
                'led_194': 50,
                'led_195': 50,
                'led_196': 50,
                'led_197': 50,
                'led_198': 50,
                'led_199': 50,
                'led_200': 50,
                'led_201': 50,
                'led_202': 50,
                'led_203': 50,
                'led_204': 50,
                'led_205': 50,
                'led_206': 50,
                'led_207': 50,
                'led_208': 50,
                'led_209': 50,
                'led_210': 50,
                'led_211': 50,
                'led_212': 50,
                'led_213': 50,
                'led_214': 50,
                'led_215': 50,
                'led_216': 50,
                'led_217': 50,
                'led_218': 50,
                'led_219': 50,
                'led_220': 50,
                'led_221': 50,
                'led_222': 50,
                'led_223': 50,
                'led_224': 50,
                'led_225': 50,
                'led_226': 50,
                'led_227': 50,
                'led_228': 50,
                'led_229': 50,
                'led_230': 50,
                'led_231': 50,
                'led_232': 50,
                'led_233': 50,
                'led_234': 50,
                'led_235': 50,
                'led_236': 50,
                'led_237': 50,
                'led_238': 50,
                'led_239': 50,
                'led_240': 50,
                'led_241': 50,
                'led_242': 50,
                'led_243': 50,
                'led_244': 50,
                'led_245': 50,
                'led_246': 50,
                'led_247': 50,
                'led_248': 50,
                'led_249': 50,
                'led_250': 50,
                'led_251': 50,
                'led_252': 50,
                'led_253': 50,
                'led_254': 50,
                'led_255': 50,
                'led_256': 50,
                'led_257': 50,
                'led_258': 50,
                'led_259': 50,
                'led_260': 50,
                'led_261': 50,
                'led_262': 50,
                'led_265': 50,
                'led_266': 50,
                'led_267': 50,
                'led_268': 50,
                'led_269': 50,
                'led_270': 50,
                'led_271': 50,
                'led_272': 50,
                'led_273': 50,
                'led_274': 50,
                'led_275': 50,
                'led_276': 50,
                'led_277': 50,
                'led_278': 50,
                'led_279': 50,
                'led_280': 50,
                'led_281': 50,
                'led_282': 50,
                'led_283': 50,
                'led_284': 50,
                'led_285': 50,
                'led_286': 50,
                'led_287': 50,
                'led_288': 50,
                'led_289': 50,
                'led_290': 50,
                'led_291': 50,
                'led_292': 50,
                'led_293': 50,
                'led_294': 50,
                'led_295': 50,
                'led_296': 50,
                'led_297': 50,
                'led_298': 50,
                'led_299': 50,
                'led_300': 50,
                'led_301': 50,
                'led_302': 50,
                'led_303': 50,
                'led_304': 50,
                'led_305': 50,
                'led_306': 50,
                'led_307': 50,
                'led_308': 50,
                'led_309': 50,
                'led_310': 50,
                'led_311': 50,
                'led_312': 50,
                'led_313': 50,
                'led_314': 50,
                'led_315': 50,
                'led_316': 50,
                'led_317': 50,
                'led_318': 50,
                'led_319': 50,
                'led_320': 50,
                'led_321': 50,
                'led_322': 50,
                'led_323': 50,
                'led_324': 50,
                'led_325': 50,
                'led_326': 50,
                'led_327': 50,
                'led_328': 50,
                'led_329': 50,
                'led_330': 50,
                'led_331': 50,
                'led_332': 50,
                'led_333': 50,
                'led_334': 50,
                'led_335': 50,
                'led_336': 50,
                'led_337': 50,
                'led_338': 50,
                'led_339': 50,
                'led_340': 50,
                'led_341': 50,
                'led_342': 50,
                'led_343': 50,
                'led_344': 50,
                'led_345': 50,
                'led_346': 50,
                'led_347': 50,
                'led_348': 50,
                'led_349': 50,
                'led_350': 50,
                'led_351': 50,
                'led_352': 50,
                'led_353': 50,
                'led_354': 50,
                'led_355': 50,
                'led_356': 50,
                'led_357': 50,
                'led_358': 50,
                'led_359': 50,
                'led_360': 50,
                'led_361': 50,
                'led_362': 50,
                'led_363': 50,
                'led_364': 50,
                'led_365': 50,
                'led_366': 50,
                'led_367': 50,
                'led_368': 50,
                'led_369': 50,
                'led_370': 50,
                'led_371': 50,
                'led_372': 50,
                'led_373': 50,
                'led_374': 50,
                'led_375': 50,
                'led_376': 50,
                'led_377': 50,
                'led_378': 50,
                'led_379': 50,
                'led_380': 50,
                'led_381': 50,
                'led_382': 50,
                'led_383': 50,
                'led_384': 50,
                'led_385': 50,
                'led_386': 50,
                'led_387': 50,
                'led_388': 50,
                'led_389': 50,
                'led_390': 50,
                'led_391': 50,
                'led_392': 50,
                'led_393': 50,
                'led_394': 50,
                'led_395': 50,
                'led_396': 50,
                'led_397': 50,
                'led_398': 50,
                'led_399': 50,
                'led_400': 50,
                'led_401': 50,
                'led_402': 50,
                'led_403': 50,
                'led_404': 50,
                'led_405': 50,
                'led_406': 50,
                'led_407': 50,
                'led_408': 50,
                'led_409': 50,
                'led_410': 50,
                'led_411': 50,
                'led_412': 50,
                'led_413': 50,
                'led_414': 50,
                'led_415': 50,
                'led_416': 50,
                'led_417': 50,
                'led_418': 50,
                'led_419': 50,
                'led_420': 50,
                'led_421': 50,
                'led_422': 50,
                'led_423': 50,
                'led_424': 50,
                'led_425': 50,
                'led_426': 50,
                'led_427': 50,
                'led_428': 50,
                'led_429': 50,
                'led_430': 50,
                'led_431': 50,
                'led_432': 50,
                'led_433': 50,
                'led_434': 50,
                'led_435': 50,
                'led_436': 50,
                'led_437': 50,
                'led_438': 50,
                'led_439': 50,
                'led_440': 50,
                'led_441': 50,
                'led_442': 50,
                'led_443': 50,
                'led_444': 50,
                'led_445': 50,
                'led_446': 50,
                'led_447': 50,
                'led_448': 50,
                'led_449': 50,
                'led_450': 50,
                'led_451': 50,
                'led_452': 50,
                'led_453': 50,
                'led_454': 50,
                'led_455': 50,
                'led_456': 50,
                'led_457': 50,
                'led_458': 50,
                'led_459': 50,
                'led_460': 50,
                'led_461': 50,
                'led_462': 50,
                'led_463': 50,
                'led_464': 50,
                'led_465': 50,
                'led_466': 50,
                'led_467': 50,
                'led_468': 50,
                'led_469': 50,
                'led_470': 50,
                'led_471': 50,
                'led_472': 50,
                'led_473': 50,
                'led_474': 50,
                'led_475': 50,
                'led_476': 50,
                'led_477': 50,
                'led_478': 50,
                'led_479': 50,
                'led_480': 50,
                'led_481': 50,
                'led_482': 50,
                'led_483': 50,
                'led_484': 50,
                'led_485': 50,
                'led_486': 50,
                'led_487': 50,
                'led_488': 50,
                'led_489': 50,
                'led_490': 50,
                'led_491': 50,
                'led_492': 50,
                'led_493': 50,
                'led_494': 50,
                'led_495': 50,
                'led_496': 50,
                'led_497': 50,
                'led_498': 50,
                'led_499': 50,
                'led_500': 50,
                'led_501': 50,
                'led_502': 50,
                'led_503': 50,
                'led_504': 50,
                'led_505': 50,
                'led_506': 50,
                'led_507': 50,
                'led_508': 50,
                'led_509': 50,
                'led_510': 50,
                'led_511': 50,
                'led_512': 50,
                'led_513': 50,
                'led_514': 50,
                'led_515': 50,
                'led_516': 50,
                'led_517': 50,
                'led_518': 50,
                'led_519': 50,
                'led_520': 50,
                'led_521': 50,
                'led_522': 50,
                'led_523': 50,
                'led_524': 50,
                'led_525': 50,
                'led_526': 50,
                'led_527': 50,
                'led_528': 50,
                'led_529': 50,
                'led_530': 50,
                'led_531': 50,
                'led_532': 50,
                'led_533': 50,
                'led_534': 50,
                'led_535': 50,
                'led_536': 50,
                'led_537': 50,
                'led_538': 50,
                'led_539': 50,
                'led_540': 50,
                'led_541': 50,
                'led_542': 50,
                'led_543': 50,
                'led_544': 50,
                'led_545': 50,
                'led_546': 50,
                'led_547': 50,
                'led_548': 50,
                'led_549': 50,
                'led_550': 50,
                'led_551': 50,
                'led_552': 50,
                'led_553': 50,
                'led_554': 50,
                'led_555': 50,
                'led_556': 50,
                'led_557': 50,
                'led_558': 50,
                'led_559': 50,
                'led_560': 50,
                'led_561': 50,
                'led_562': 50,
                'led_563': 50,
                'led_564': 50,
                'led_565': 50,
                'led_566': 50,
                'led_567': 50,
                'led_568': 50,
                'led_569': 50,
                'led_570': 50,
                'led_571': 50,
                'led_572': 50,
                'led_573': 50,
                'led_574': 50,
                'led_575': 50,
                'led_576': 50,
                'led_577': 50,
                'led_578': 50,
                'led_579': 50,
                'led_580': 50,
                'led_581': 50,
            },
            'wavelength_nm': 515,
            'pixel_size_um': 0.166,
            'objective': {'NA': 0.5, 'magnification': 20}
        },
        'fluorescence': {
            'channel_names': [fl_channel_renaming[f] for f in fluor_channel_names],
            'channel_indices': {fl_channel_renaming[ch_name]: i for i, ch_name
                                in enumerate(fluor_channel_names)},
            'exposure_ms': {
                fl_channel_renaming['F1_BV711']: 1600,
                fl_channel_renaming['F2_BV650']: 2000,
                fl_channel_renaming['F3_BV605']: 1500,
                fl_channel_renaming['F4_BV570']: 1200,
                fl_channel_renaming['F5_BV510']: 1000,
                fl_channel_renaming['F6_BV421']: 600
            },
            'emission_wavelength_nm': {
              'Fluor_690-': [690, None],
              'Fluor_627-673': [627, 673],
              'Fluor_585-625': [585, 625],
              'Fluor_550-570': [550, 570],
              'Fluor_500-550': [500, 550],
              'Fluor_426-446': [426, 446],
            },
            'excitation_wavelength_nm': 405,
            'pixel_size_um': 0.166,
            'objective': {'NA': 0.5, 'magnification': 20}
        },
    }


    with open(export_root + '/BSCCM-coherent_global_metadata.json', 'w') as json_file:
        json.dump(metadata, json_file)